In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd; pd.options.mode.chained_assignment = None

import APIs.Bloomberg as ba
import Weather.W_USA as wu

import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import plotly.express as px

In [3]:
blp = ba.BLPInterface('//blp/exrsvc')

In [15]:
run = dt(2022,7,5,6,0,0)
run_str = run.strftime("%Y-%m-%dT%H:%M:%S")

print(run_str)

location = 'US_IA'
forecast = 'ECMWF'
forecast = 'GFS'
# overrides = {'location': location, 'fields':'TEMPERATURE|PRECIPITATION', 'model':forecast,'publication_date':'2022-07-04T00:00:00','location_time':True}
overrides = {'location': location, 'fields':'TEMPERATURE|PRECIPITATION', 'model':forecast,'publication_date':run_str,'location_time':True}

df_GFS = blp.bsrch('comdty:weather', overrides)
df_GFS['Location Time'] =  pd.to_datetime(df_GFS['Location Time'])
df_GFS['date']=df_GFS['Location Time'].dt.date
df_GFS['Precipitation (mm)'].iloc[0]=0
df_GFS

2022-07-05T06:00:00


,Reported Time,Location Time,Temperature (°C),Precipitation (mm),date
0,2022-07-05 06:00:00+00:00,2022-07-05 01:00:00-05:00,25.91,0,2022-07-05
1,2022-07-05 09:00:00+00:00,2022-07-05 04:00:00-05:00,24.19,0.02,2022-07-05
2,2022-07-05 12:00:00+00:00,2022-07-05 07:00:00-05:00,24.16,0.0,2022-07-05
3,2022-07-05 15:00:00+00:00,2022-07-05 10:00:00-05:00,29.54,0.02,2022-07-05
4,2022-07-05 18:00:00+00:00,2022-07-05 13:00:00-05:00,32.87,0.0,2022-07-05
...,...,...,...,...,...
124,2022-07-20 18:00:00+00:00,2022-07-20 13:00:00-05:00,31.29,0.0,2022-07-20
125,2022-07-20 21:00:00+00:00,2022-07-20 16:00:00-05:00,33.42,0.04,2022-07-20
126,2022-07-21 00:00:00+00:00,2022-07-20 19:00:00-05:00,31.20,1.84,2022-07-20
127,2022-07-21 03:00:00+00:00,2022-07-20 22:00:00-05:00,24.46,1.19,2022-07-20


In [ ]:
prec_df = df_GFS.groupby('date')[['Precipitation (mm)']].sum()
type(prec_df)

In [ ]:
corn_states=['IA','IL']
run=dt(2022,7,4,12,0,0)
wu.udpate_USA_Bloomberg(run, corn_states, forecast='GFS')